In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

# Remplacez ces valeurs par les informations de votre base de données
db_host = "localhost"
db_name = "olist"
db_user = "postgres"
db_password = "0000"

# Créer une connexion avec la base de données en utilisant sqlalchemy
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")


# Customer

In [2]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_customers_dataset.csv", encoding="utf-8")
data.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [3]:
zip_codes = data[['customer_zip_code_prefix']].drop_duplicates()
zip_codes.columns = ['zip_code_prefix']

states = data[['customer_state']].drop_duplicates()
states.columns = ['state_code']

cities = data[['customer_city', 'customer_state']].drop_duplicates()
cities.columns = ['city_name', 'state_code']

city_zip_codes = data[['customer_city', 'customer_state', 'customer_zip_code_prefix']].drop_duplicates()
city_zip_codes.columns = ['city_name', 'state_code', 'zip_code_prefix']

customers = data[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix']].drop_duplicates()
customers.columns = ['customer_id', 'customer_unique_id', 'zip_code_prefix']

zip_codes.to_sql('olist_zipcode', engine, if_exists='append', index=False)
states.to_sql('olist_state', engine, if_exists='append', index=False)
cities.to_sql('olist_city', engine, if_exists='append', index=False)
customers.to_sql('olist_customer', engine, if_exists='append', index=False)

# Récupérer les city_id à partir de la base de données
cities_db = pd.read_sql("SELECT city_id, city_name, state_code FROM olist_city", engine)

# Joindre les DataFrames
city_zip_codes_joined = city_zip_codes.merge(cities_db, on=['city_name', 'state_code'])

# Insérer les données dans la table city_zip_code
city_zip_codes_joined[['city_id', 'zip_code_prefix']].to_sql('olist_cityzipcode', engine, if_exists='append', index=False)

34

# Orders

In [4]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_orders_dataset.csv", encoding="utf-8")
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
date_columns = ['order_approved_at', 'order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for column in date_columns:
    data[column] = pd.to_datetime(data[column])

data.to_sql('olist_order', engine, if_exists='append', index=False)

441

# Order Payments

In [6]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_order_payments_dataset.csv", encoding="utf-8")
data.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [7]:
data.to_sql('olist_orderpayment', engine, if_exists='append', index=False)

886

# Order Reviews

In [8]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_order_reviews_dataset.csv", encoding="utf-8")
data.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [9]:
data.to_sql('olist_orderreview', engine, if_exists='append', index=False)

224

# Product Category Name Traductions

In [10]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/product_category_name_translation.csv", encoding="utf-8")
data.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [11]:
data.to_sql('olist_productcategorynametranslation', engine, if_exists='append', index=False)

71

# Products

In [12]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_products_dataset.csv", encoding="utf-8")
data.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [13]:
data = data.rename(columns={"product_category_name": "product_category_name_translation"})
data["product_category_name_translation"].fillna("unknown_category", inplace=True)

unique_categories = set(data["product_category_name_translation"].dropna().unique())
existing_categories = pd.read_sql("SELECT product_category_name FROM olist_productcategorynametranslation", engine)
missing_categories = unique_categories - set(existing_categories["product_category_name"].values)

if missing_categories:
    missing_categories_df = pd.DataFrame({"product_category_name": list(missing_categories),
                                          "product_category_name_english": ["unknown"] * len(missing_categories)})
    missing_categories_df.to_sql('olist_productcategorynametranslation', engine, if_exists='append', index=False)

data.to_sql('olist_product', engine, if_exists='append', index=False)

951

# Sellers

In [14]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_sellers_dataset.csv", encoding="utf-8")
data.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [15]:
# Ajouter les codes postaux manquants
unique_zip_codes = data['seller_zip_code_prefix'].drop_duplicates().to_frame()
unique_zip_codes.columns = ['zip_code_prefix']
unique_zip_codes['zip_code_prefix'] = unique_zip_codes['zip_code_prefix'].astype(str)

existing_zip_codes = pd.read_sql("SELECT zip_code_prefix FROM olist_zipcode", engine)
existing_zip_codes['zip_code_prefix'] = existing_zip_codes['zip_code_prefix'].astype(str)

# Fusionner les deux dataframes pour trouver les codes postaux manquants
missing_zip_codes = unique_zip_codes.merge(existing_zip_codes, on='zip_code_prefix', how='left', indicator=True)
missing_zip_codes = missing_zip_codes[missing_zip_codes['_merge'] == 'left_only'][['zip_code_prefix']]

# Ajouter les codes postaux manquants à la table olist_zipcode
missing_zip_codes.to_sql('olist_zipcode', engine, if_exists='append', index=False)

# Ajouter les états manquants
unique_states = data['seller_state'].drop_duplicates()
states_db = pd.read_sql("SELECT state_code FROM olist_state", engine)
missing_states = unique_states.loc[~unique_states.isin(states_db['state_code'])]
missing_states = pd.DataFrame(missing_states, columns=['state_code'])
missing_states.to_sql('olist_state', engine, if_exists='append', index=False)

# Ajouter les villes manquantes
unique_cities = data[['seller_city', 'seller_state']].drop_duplicates()
cities_db = pd.read_sql("SELECT city_name, state_code FROM olist_city", engine)
missing_cities = unique_cities.merge(cities_db, left_on=['seller_city', 'seller_state'], right_on=['city_name', 'state_code'], how='left', indicator=True)
missing_cities = missing_cities[missing_cities['_merge'] == 'left_only'][['seller_city', 'seller_state']]
missing_cities.columns = ['city_name', 'state_code'] # Renommer les colonnes pour correspondre à la table city
missing_cities.to_sql('olist_city', engine, if_exists='append', index=False)

# Maintenant, ajouter les relations manquantes entre les villes et les codes postaux
cities_db = pd.read_sql("SELECT city_id, city_name, state_code FROM olist_city", engine)
city_zip_codes = data[['seller_city', 'seller_state', 'seller_zip_code_prefix']].drop_duplicates()
city_zip_codes_joined = city_zip_codes.merge(cities_db, left_on=['seller_city', 'seller_state'], right_on=['city_name', 'state_code'])
city_zip_codes_joined = city_zip_codes_joined[['city_id', 'seller_zip_code_prefix']]
city_zip_codes_joined.columns = ['city_id', 'zip_code_prefix']  # Renommer les colonnes pour correspondre à la table city_zip_code

existing_city_zip_codes = pd.read_sql("SELECT city_id, zip_code_prefix FROM olist_cityzipcode", engine)

# Convertir les types de données pour qu'ils correspondent
city_zip_codes_joined['city_id'] = city_zip_codes_joined['city_id'].astype(int)
city_zip_codes_joined['zip_code_prefix'] = city_zip_codes_joined['zip_code_prefix'].astype(str)
existing_city_zip_codes['city_id'] = existing_city_zip_codes['city_id'].astype(int)
existing_city_zip_codes['zip_code_prefix'] = existing_city_zip_codes['zip_code_prefix'].astype(str)

city_zip_codes_to_insert = city_zip_codes_joined.merge(existing_city_zip_codes, on=['city_id', 'zip_code_prefix'], how='left', indicator=True)
city_zip_codes_to_insert = city_zip_codes_to_insert[city_zip_codes_to_insert['_merge'] == 'left_only'][['city_id', 'zip_code_prefix']]

city_zip_codes_to_insert.to_sql('olist_cityzipcode', engine, if_exists='append', index=False)

# Insérer les données des vendeurs dans la table Seller
sellers_data = data[['seller_id', 'seller_zip_code_prefix']].copy()
sellers_data.columns = ['seller_id', 'zip_code_prefix']  # Renommer la colonne pour correspondre au modèle
sellers_data.to_sql('olist_seller', engine, if_exists='append', index=False)

95